In [6]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import regex as rex
import date_library as datelib
import utils as u

In [8]:
try:
    # df = pd.read_csv("activity_reduced_before_attributes.csv")
    df = pd.read_csv("member_d.csv")
except:
    # df = pd.read_csv("activity_reduced_before_attributes.csv.gz")
    df = pd.read_csv("member_d.csv.gz")
    

In [9]:
df.columns, df.shape

(Index(['MEMBER_ID', 'TRUE_OD', 'D', 'FLIGHT_DATE', 'BOOKING_DATE',
        'TICKET_SALES_DATE', 'TICKET_NUMBER', 'TRUE_ORIGIN_COUNTRY',
        'ADDR_COUNTRY', 'PNR', 'PARTY_SZ', 'size'],
       dtype='object'),
 (835129, 12))

# Create month and day of week columns for Booking date and flight date
Column names: book_mo, book_dofwk, flight_mo, flight_daywk

In [10]:
df1 = u.get_dayofweek_month(df, df_date_col='BOOKING_DATE', 
        labels=('booking_date', 'booking_mo', 'booking_dowk'))

In [11]:
df2 = u.get_dayofweek_month(df1, df_date_col='FLIGHT_DATE', 
        labels=('flight_date', 'flight_mo', 'flight_dowk'))

In [12]:
df2.columns

Index(['MEMBER_ID', 'TRUE_OD', 'D', 'FLIGHT_DATE', 'BOOKING_DATE',
       'TICKET_SALES_DATE', 'TICKET_NUMBER', 'TRUE_ORIGIN_COUNTRY',
       'ADDR_COUNTRY', 'PNR', 'PARTY_SZ', 'size', 'booking_date',
       'booking_dowk', 'booking_mo', 'flight_date', 'flight_dowk',
       'flight_mo'],
      dtype='object')

In [13]:
dfm = pd.read_csv("MEMBERS_CATALOG.csv")
dfm.columns

Index(['MEMBER_ID', 'TIER_LEVEL', 'TIER_LEVEL_DESCRIPTION', 'PREVIOUS_TIER',
       'LAST_TIER_CHANGE_DATE', 'STATUS', 'ENROLL_DATE', 'GENDER',
       'BIRTH_DATE', 'NATIONALITY', 'ADDR_COUNTRY', 'COMMUNICATION_LANGUAGE',
       'CONNECTMILES_OFFERS_AND_NEWS', 'E_STATEMENTS',
       'COPA_AIRLINES_OFFERS_AND_NEWS', 'CCD_ACTIVE_IND', 'MILES_BALANCE',
       'TOTAL_MILES_SINCE_ENROLLMENT'],
      dtype='object')

In [20]:
dfm = dfm[['MEMBER_ID', 'TIER_LEVEL', 'GENDER', 'BIRTH_DATE']]

In [21]:
dfm1 = dfm[pd.isna(dfm['GENDER'])]

In [22]:
dfm.BIRTH_DATE

0        1962-02-14
1        1979-01-16
2        1970-12-21
3        1974-07-31
4        1988-09-21
            ...    
96840    1964-11-05
96841    1980-11-28
96842    1983-08-03
96843    1973-09-15
96844    1985-05-05
Name: BIRTH_DATE, Length: 96845, dtype: object

In [23]:
import datetime
today = datetime.date.today().strftime("%Y-%m-%d")
today1 = datelib.dateTimePTYToTimestamp(today)

In [24]:
today1 

1655823600

# Compute passenger age when the flight departed. 

In [27]:
x = pd.merge(df2, dfm, how='inner', on='MEMBER_ID')

In [28]:
x.columns

Index(['MEMBER_ID', 'TRUE_OD', 'D', 'FLIGHT_DATE', 'BOOKING_DATE',
       'TICKET_SALES_DATE', 'TICKET_NUMBER', 'TRUE_ORIGIN_COUNTRY',
       'ADDR_COUNTRY', 'PNR', 'PARTY_SZ', 'size', 'booking_date',
       'booking_dowk', 'booking_mo', 'flight_date', 'flight_dowk', 'flight_mo',
       'TIER_LEVEL', 'GENDER', 'BIRTH_DATE'],
      dtype='object')

In [36]:
birthdate = datelib.date_to_timestamp(x['BIRTH_DATE'])

In [37]:
x['age_at_flight'] = (x['FLIGHT_DATE'] - birthdate) / (365*24*3600)
x['age_at_flight'] = x.age_at_flight.apply(int)
# pd.isna(x['BOOKING_DATE']).sum()
#x['age_at_flight'] = x['age_at_flight'].astype(int)

In [38]:
x['age_at_flight']

0         56
1         56
2         56
3         56
4         54
          ..
835124    30
835125    36
835126    32
835127    62
835128    63
Name: age_at_flight, Length: 835129, dtype: int64

In [39]:
x.columns

Index(['MEMBER_ID', 'TRUE_OD', 'D', 'FLIGHT_DATE', 'BOOKING_DATE',
       'TICKET_SALES_DATE', 'TICKET_NUMBER', 'TRUE_ORIGIN_COUNTRY',
       'ADDR_COUNTRY', 'PNR', 'PARTY_SZ', 'size', 'booking_date',
       'booking_dowk', 'booking_mo', 'flight_date', 'flight_dowk', 'flight_mo',
       'TIER_LEVEL', 'GENDER', 'BIRTH_DATE', 'age_at_flight'],
      dtype='object')

In [40]:
# Party size: 9 values
# Need trip duration. 

In [41]:
attribute_cols = ['MEMBER_ID', 'booking_dowk', 'booking_mo', 'flight_dowk', 
                  'flight_mo', 'age_at_flight', 'PARTY_SZ']
attributes = x[attribute_cols]

# 67 passenges are 2 years old? Not possible! Check this out. 

In [42]:
pax_age2 = attributes[attributes['age_at_flight'] == 2].head()
pax_age2

,MEMBER_ID,booking_dowk,booking_mo,flight_dowk,flight_mo,age_at_flight,PARTY_SZ
5793,233753592,3,3,2,4,2,1
5794,233753592,5,3,1,4,2,1
5795,233753592,3,1,3,3,2,1
5796,233753592,3,1,1,3,2,1
170580,242778655,3,1,4,8,2,1


In [43]:
attributes.age_at_flight.value_counts().sort_index().head(50);

In [44]:
pax_age2.booking_mo.value_counts().sort_index()
x.booking_dowk.value_counts().sort_index()

0    179516
1    145944
2    142122
3    138827
4    130237
5     53409
6     45074
Name: booking_dowk, dtype: int64

In [45]:
df[df['MEMBER_ID'] == 230831718][['MEMBER_ID', 'FLIGHT_DATE']].head(10)

,MEMBER_ID,FLIGHT_DATE
271579,230831718,1557187200
271580,230831718,1622332800
271581,230831718,1467849600
271582,230831718,1564272000
271583,230831718,1460160000
271584,230831718,1564963200
271585,230831718,1498003200
271586,230831718,1510012800
271587,230831718,1547683200
271588,230831718,1501372800


In [46]:
dfm[dfm['MEMBER_ID'] == 230831718]

,MEMBER_ID,TIER_LEVEL,GENDER,BIRTH_DATE
10436,230831718,T3,M,2013-03-03


In [47]:
# Birth date: 2013-03-03
# Flight date: 1510012800

datelib.timestampToDateTimePTY(1510012800)

('2017-11-07', '05:00')

Age: 4 years old. 

In [48]:
dfull = u.read_activity_file()
dfull[dfull['MEMBER_ID'] == 230831718][['MEMBER_ID', 'FLIGHT_DATE']].head(10)

/home/erlebach/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3357: DtypeWarning: Columns (37,46) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


,MEMBER_ID,FLIGHT_DATE
2010715,230831718,NaN
2010716,230831718,NaN
2010717,230831718,2015-08-02
2010718,230831718,NaN
2010719,230831718,2015-08-06
2010720,230831718,2015-09-12
2010721,230831718,2015-09-20
2010722,230831718,2015-09-27
2010723,230831718,2015-12-01
2010724,230831718,2015-12-03


In [53]:
dfull[dfull['MEMBER_ID'] == 230831718][['MEMBER_ID', 'FLIGHT_DATE']].head(10)

,MEMBER_ID,FLIGHT_DATE
2010715,230831718,NaN
2010716,230831718,NaN
2010717,230831718,2015-08-02
2010718,230831718,NaN
2010719,230831718,2015-08-06
2010720,230831718,2015-09-12
2010721,230831718,2015-09-20
2010722,230831718,2015-09-27
2010723,230831718,2015-12-01
2010724,230831718,2015-12-03


In [54]:
dfm.sort_values('BIRTH_DATE', ascending=False).head(10)

,MEMBER_ID,TIER_LEVEL,GENDER,BIRTH_DATE
95185,268505626,T1,F,2019-11-22
92244,265498544,T1,NaN,2019-08-28
95491,269085786,T1,NaN,2019-08-22
92857,265995448,T1,M,2019-08-03
92109,265424727,T1,M,2019-05-19
94942,268195062,T1CC,M,2019-05-09
93840,266834074,T1,NaN,2019-04-02
94810,268010343,T1,F,2019-01-15
88036,261747712,T1,M,2019-01-13
92198,265469877,T1,NaN,2018-11-20


In [55]:
x.columns

Index(['MEMBER_ID', 'TRUE_OD', 'D', 'FLIGHT_DATE', 'BOOKING_DATE',
       'TICKET_SALES_DATE', 'TICKET_NUMBER', 'TRUE_ORIGIN_COUNTRY',
       'ADDR_COUNTRY', 'PNR', 'PARTY_SZ', 'size', 'booking_date',
       'booking_dowk', 'booking_mo', 'flight_date', 'flight_dowk', 'flight_mo',
       'TIER_LEVEL', 'GENDER', 'BIRTH_DATE', 'age_at_flight'],
      dtype='object')

In [56]:
attributes.columns

Index(['MEMBER_ID', 'booking_dowk', 'booking_mo', 'flight_dowk', 'flight_mo',
       'age_at_flight', 'PARTY_SZ'],
      dtype='object')

In [57]:
x.to_csv("activity_reduced_with_attributes.csv", index=0)

In [58]:
attributes.columns

Index(['MEMBER_ID', 'booking_dowk', 'booking_mo', 'flight_dowk', 'flight_mo',
       'age_at_flight', 'PARTY_SZ'],
      dtype='object')

In [59]:
x.shape, x.columns

((835129, 22),
 Index(['MEMBER_ID', 'TRUE_OD', 'D', 'FLIGHT_DATE', 'BOOKING_DATE',
        'TICKET_SALES_DATE', 'TICKET_NUMBER', 'TRUE_ORIGIN_COUNTRY',
        'ADDR_COUNTRY', 'PNR', 'PARTY_SZ', 'size', 'booking_date',
        'booking_dowk', 'booking_mo', 'flight_date', 'flight_dowk', 'flight_mo',
        'TIER_LEVEL', 'GENDER', 'BIRTH_DATE', 'age_at_flight'],
       dtype='object'))